In [1]:
from pprint import pprint

from pvsamlab.system import System 

In [2]:
for modules_per_string in range(25, 31):
    print(f"Running simulation for modules_per_string={modules_per_string}")
    plant = System(modules_per_string=modules_per_string)
    pprint(plant.model_results)
    print("-" * 50)

Running simulation for modules_per_string=25

Starting data download for solar using 1 thread workers.
Getting list of available NSRDB files for 32.71595, -101.91084.
List of available data saved to /Users/ihorkoshkin/Library/Mobile Documents/com~apple~CloudDocs/Documents/jupyter/pvsamlab/pvsamlab/data/weather_files/32.7159_-101.9108/tmy/nsrdb_data_query_response_32.71595_-101.91084.json.
https://developer.nrel.gov/api/nsrdb/v2/solar/nsrdb-GOES-tmy-v4-0-0-download.csv?names=tmy&wkt=POINT%28-101.91084+32.71595%29&interval=60&api_key=CDO5hSAzjPS5Vrb3uyKrTw5MqDLfhyZdsr8zNaVN&email=ihorkoshkin@gmail.com&utc=false
Success! File downloaded to /Users/ihorkoshkin/Library/Mobile Documents/com~apple~CloudDocs/Documents/jupyter/pvsamlab/pvsamlab/data/weather_files/32.7159_-101.9108/tmy/nsrdb_32.71595_-101.91084_nsrdb-GOES-tmy-v4-0-0_60_tmy.csv.
{'ac_annual': 304115327.721309, 'voc_max': 1271.6493225097654}
--------------------------------------------------
Running simulation for modules_per_strin

In [ ]:
from pvsamlab.climate import download_nsrdb_csv

download_nsrdb_csv(
    lat=30.9759,
    lon=-97.2465,
    start_year=2022,
    end_year=2022,
    frequency="hourly",
    time_zone="America/Los_Angeles",
    output_dir=".",
)

In [ ]:
chillingham = System(lat=30.9759, lon=-97.2465, met_year='tmy')
print(chillingham.model_results)


In [ ]:
for year in range(1998, 2024):
    print(f"Running simulation for year={year}")
    plant = System(lat=30.9759, lon=-97.2465, met_year=year)
    pprint(plant.model_results)
    print("-" * 50)